<a href="https://colab.research.google.com/github/jinkyukim-me/Image-classification-pytorch/blob/master/image_classification_catsdogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [158]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torchsummary import summary 
import math
import numpy as np

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
#parameters
learning_rate = 0.02
training_epochs = 30
batch_size = 100

In [0]:
trans = transforms.Compose([transforms.Resize((224, 224)), 
                            transforms.ToTensor(),
                            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_data = dsets.ImageFolder(root='/content/gdrive/My Drive/project/classification/catsdogs/data/train_data',
                               transform=trans)

In [0]:
#Dataloader
data_loader = DataLoader(dataset = train_data, batch_size = batch_size,
                         shuffle = True, num_workers=2, drop_last=True)

In [0]:
#model CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 5)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32*53*53, 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, 2)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1, 32*53*53)
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.softmax(self.fc3(x))
        return x


In [0]:
model = CNN().to(device)

In [0]:
test_input = (torch.Tensor(3,3,224,224)).to(device)
test_out = model(test_input)

In [0]:
#loss function
criterion = nn.CrossEntropyLoss().to(device)

In [0]:
#optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum= 0.9)

In [190]:
#training
total_batch = len(data_loader)
print('Learning Started. It will takes some time.')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for num, data in enumerate(data_loader): 
        imgs, labels = data
        imgs = imgs.to(device) #input data
        labels = labels.to(device) #label data
        
        optimizer.zero_grad()
        hypothesis = model(imgs) #model에 input값을 넣으면 가설이 됨
        
        cost = criterion(hypothesis, labels) #가설과 실제 라벨 사이의 차이를 계산하는 loss
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch #cost값을 쌓아줌
        
    print('[Epoch:{}] cost = {}'.format(epoch +1, avg_cost))
print('Learning Finished!')

Learning Started. It will takes some time.
[Epoch:1] cost = 0.6964457631111145
[Epoch:2] cost = 0.6663315296173096
[Epoch:3] cost = 0.646409809589386
[Epoch:4] cost = 0.5964276194572449
[Epoch:5] cost = 0.5733487010002136
[Epoch:6] cost = 0.5851479768753052
[Epoch:7] cost = 0.4938213527202606
[Epoch:8] cost = 0.37592947483062744
[Epoch:9] cost = 0.2662104666233063
[Epoch:10] cost = 0.1833043396472931
[Epoch:11] cost = 0.11438131332397461
[Epoch:12] cost = 0.09056733548641205
[Epoch:13] cost = 0.07484141737222672
[Epoch:14] cost = 0.043094296008348465
[Epoch:15] cost = 0.06640555709600449
[Epoch:16] cost = 0.02583288960158825
[Epoch:17] cost = 0.01825803704559803
[Epoch:18] cost = 0.01825612224638462
[Epoch:19] cost = 0.01665448769927025
[Epoch:20] cost = 0.03359825536608696
[Epoch:21] cost = 0.030917005613446236
[Epoch:22] cost = 0.016622275114059448
[Epoch:23] cost = 0.004858820699155331
[Epoch:24] cost = 0.0031131329014897346
[Epoch:25] cost = 0.010382309556007385
[Epoch:26] cost = 0

In [0]:
trans = transforms.Compose([transforms.Resize((224, 224)), 
                            transforms.ToTensor(), 
                            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_data = dsets.ImageFolder(root='/content/gdrive/My Drive/project/classification/catsdogs/data/test_data',
                               transform=trans)

In [0]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [181]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_set:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of CNN on the test images: %d %%' % (
    100 * correct / total))

Accuracy of CNN on the test images: 68 %
